In [1]:
from math import sqrt
import numpy as np
import pandas as pd
import pickle

from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import Dropout

2023-01-12 20:08:10.889470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 20:08:11.233560: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-12 20:08:11.233608: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-12 20:08:12.687522: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# 数据集特征
n_back = 200
n_out = 16
n_pre = n_out*15*2
n_feature = 2

train_day = 6
validation_day = 4
test_day = 3

# 神经网络参数
import numpy as np
units = np.arange(20, 60, 10)
batch_size = 32
epochs = 50
units

array([20, 30, 40, 50])

In [3]:
dataset = pd.read_csv('../wind_preprocessed.csv', header=0, index_col=0).query('day<14')
data = dataset[['speed_moveavg', 'power_moveavg']].rename(columns={'power_moveavg':'power', 'speed_moveavg':'speed'})
print(data.head())

                       speed    power
date                                 
2015-10-01 00:00:00  0.39627  0.38065
2015-10-01 00:00:30  0.39592  0.36943
2015-10-01 00:01:00  0.39538  0.38529
2015-10-01 00:01:30  0.39579  0.38892
2015-10-01 00:02:00  0.39627  0.41220


In [4]:
values = data.values
values = values.astype('float32')

def series_to_supervised(data, n_in, n_out, colname, dropnan=True):
    n_vars = colname
    # n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in n_vars]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out, 15*2):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (j)) for j in n_vars]
        else:
            names += [('%s(t+%d)' % (j, i)) for j in n_vars]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# 构建成监督学习问题
reframed = series_to_supervised(values, n_back, n_pre, ['speed', 'power'])
# 丢弃我们不想预测的列
for i in range(0, n_pre, 15*2):
    if i == 0:
        colname = 'speed(t)'
    else:
        colname = f'speed(t+{i})'
    reframed.drop(colname, axis=1, inplace=True)
# print(reframed.head(5))

In [5]:
# 分割为训练集和测试集
values = reframed.values
n_train = train_day*24*60*2
n_validation = validation_day*24*60*2
n_test = test_day*24*60*2
train = values[:n_train, :]
validation = values[n_train:n_train+n_validation, :]
test = values[-n_test:, :]
# 分为输入输出
n_obs = n_back * n_feature
train_X, train_y = train[:, :n_obs], train[:, -n_out:]
validation_X, validation_y = validation[:, :n_obs], validation[:, -n_out:]
test_X, test_y = test[:, :n_obs], test[:, -n_out:]
# 重塑成3D格式 [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_back, n_feature))
validation_X = validation_X.reshape((validation_X.shape[0], n_back, n_feature))
test_X = test_X.reshape((test_X.shape[0], n_back, n_feature))
print(train_X.shape, train_y.shape)
print(validation_X.shape, validation_y.shape)
print(test_X.shape, test_y.shape)

(17280, 200, 2) (17280, 16)
(11520, 200, 2) (11520, 16)
(8640, 200, 2) (8640, 16)


In [6]:
# model_m = Sequential()
# # model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
# model_m.add(Conv1D(100, 10, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
# model_m.add(Conv1D(100, 10, activation='relu'))
# model_m.add(MaxPooling1D(3))
# model_m.add(Conv1D(160, 10, activation='relu'))
# model_m.add(Conv1D(160, 10, activation='relu'))
# model_m.add(GlobalAveragePooling1D())
# model_m.add(Dropout(0.5))
# model_m.add(Dense(n_out, activation='softmax'))
# print(model_m.summary())

2023-01-12 20:08:16.570694: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-12 20:08:16.570944: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-12 20:08:16.571024: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-VC833VJB): /proc/driver/nvidia/version does not exist
2023-01-12 20:08:16.572131: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 191, 100)          2100      
                                                                 
 conv1d_1 (Conv1D)           (None, 182, 100)          100100    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 60, 100)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 51, 160)           160160    
                                                                 
 conv1d_3 (Conv1D)           (None, 42, 160)           256160    
                                                                 
 global_average_pooling1d (G  (None, 160)              0         
 lobalAveragePooling1D)                                 

In [8]:
sw_width = 200
n_features = 1
epoch_num = 500
verbose_set = 1

In [12]:
from keras.layers import  Dense,Dropout,Flatten
model = Sequential()
    
# 对于一维卷积来说，data_format='channels_last'是默认配置，该API的规则如下：
# 输入形状为：(batch, steps, channels)；输出形状为：(batch, new_steps, filters)，padding和strides的变化会导致new_steps变化
# 如果设置为data_format = 'channels_first'，则要求输入形状为： (batch, channels, steps).
model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(sw_width, n_features)))
model.add(Dropout((0.5)))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(sw_width, n_features)))

# model.add(Dropout((0.5)))

# model.add(Dense(units=50, activation='relu',
#                 use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))

# 对于一维池化层来说，data_format='channels_last'是默认配置，该API的规则如下：
# 3D 张量的输入形状为: (batch_size, steps, features)；输出3D张量的形状为：(batch_size, downsampled_steps, features)
# 如果设置为data_format = 'channels_first'，则要求输入形状为：(batch_size, features, steps)
model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
# data_format参数的作用是在将模型从一种数据格式切换到另一种数据格式时保留权重顺序。默认为channels_last。
# 如果设置为channels_last，那么数据输入形状应为：（batch，…，channels）；如果设置为channels_first，那么数据输入形状应该为（batch，channels，…）
# 输出为（batch, 之后参数尺寸的乘积）
     
model.add(Flatten())

# Dense执行以下操作：output=activation（dot（input，kernel）+bias），
# 其中,activation是激活函数，kernel是由层创建的权重矩阵，bias是由层创建的偏移向量（仅当use_bias为True时适用）。
# 2D 输入：(batch_size, input_dim)；对应 2D 输出：(batch_size, units)
model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
model.add(Dropout((0.5)))
# 因为要预测下一个时间步的值，因此units设置为1
model.add(Dense(units=1))
    
# 配置模型
model.compile(optimizer='adam', loss='mse',
                  loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
print('\n',model.summary())
# X为输入数据，y为数据标签；batch_size：每次梯度更新的样本数，默认为32。
# verbose: 0,1,2. 0=训练过程无输出，1=显示训练过程进度条，2=每训练一个epoch打印一次信息
    
history = model.fit(train_X, train_y, batch_size=400, epochs=epoch_num, verbose=verbose_set,validation_data=(validation_X, validation_y))
# 保存模型及训练历史
import pickle
model.save_weights(f'CNN_1.h5')
with open(f'CNN_1_hist.pickle', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_10 (Conv1D)          (None, 199, 64)           192       
                                                                 
 dropout_6 (Dropout)         (None, 199, 64)           0         
                                                                 
 conv1d_11 (Conv1D)          (None, 197, 64)           12352     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 98, 64)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 6272)              0         
                                                                 
 dense_5 (Dense)             (None, 50)                313650    
                                                      

ValueError: in user code:

    File "/home/ellahu/anaconda3/envs/bigdata/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/ellahu/anaconda3/envs/bigdata/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ellahu/anaconda3/envs/bigdata/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/ellahu/anaconda3/envs/bigdata/lib/python3.8/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/ellahu/anaconda3/envs/bigdata/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/ellahu/anaconda3/envs/bigdata/lib/python3.8/site-pac